In [11]:
# experimental data


import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
from scipy import stats
from scipy.linalg import inv, eig

sampling_rate = 128
num_samples = 100
time = np.arange(0,(num_samples)/sampling_rate, 1/sampling_rate)
base_sin8 =  np.sin(2*np.pi*time*8)
base_sin10 = np.sin(2*np.pi*time*10)
base_sin12 =  np.sin(2*np.pi*time*12)
base_sin15 = np.sin(2*np.pi*time*15)
base_sin20 = np.sin(2*np.pi*time*20)

## The input matrix 
# ! this will be just the input for the function
sin_noise10 = base_sin10 + .2*np.random.randn(num_samples)
sin_noise12 = base_sin12 + .2*np.random.randn(num_samples)
sin_noise15 = base_sin15 + .2*np.random.randn(num_samples)

X1 = sin_noise12
X2 = 10*sin_noise12
X3 = sin_noise12

X_input = np.stack(((X1, X2, X3))).T

In [7]:
f1 = 10
f2 = 12
f3 = 15

In [8]:
import numpy as np
from scipy import stats, integrate
#import scipy.fftpack
from scipy.signal import butter, lfilter, filtfilt
from scipy.linalg import inv, eig


# Filters
def butter_bandpass(lowcut, highcut, fs, order=8):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filtfilt(data, lowcut, highcut, fs, order=8):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return filtfilt(b, a, data) 


# CCA function
def CCA_corrcoeff(X, Y):
        """Function calculates correlations coeffciencts r"""
        Z = np.column_stack((X, Y,))
        C = np.cov(Z.T)

        sy = np.shape(Y)[1] if Y.ndim > 1 else 1
        sx = np.shape(X)[1] if X.ndim > 1 else 1

        Cxx = C[0:sx, 0:sx] + 10**(-8) * np.eye(sx)
        Cxy = C[0:sx, sx:sx+sy]
        Cyx = Cxy.T
        Cyy = C[sx:sx+sy, sx:sx+sy] + 10**(-8) * np.eye(sy)
        invCyy = inv(Cyy)
        invCxx = inv(Cxx)

        r, Wx = eig( np.dot( np.dot(invCxx, Cxy), np.dot(invCyy, Cyx) ) )
        r = np.real(r)
        r_sqrt = np.sqrt(np.round(r, 7))

        return r_sqrt

In [12]:
    # BPF the input X (each electrode seperately)
    X = np.zeros_like(X_input)
    for i in range(X_input.shape[1]):
        X[:,i] = butter_bandpass_filtfilt(X_input[:,i], (np.min([f1,f2,f3])-5), (np.max([f1,f2,f3])+5), sampling_rate)
    
    # Set up the targets Y
    num_samples = X.shape[0]
    time = np.arange(0, (num_samples)/sampling_rate, 1/sampling_rate)
    
    # Target frequency and harmonics  
    Y1 = np.stack(((np.sin(2*np.pi*time*f1)  , np.cos(2*np.pi*time*f1),
                    np.sin(2*np.pi*time*f1*2), np.cos(2*np.pi*time*f1*2),
                    np.sin(2*np.pi*time*f1*4), np.cos(2*np.pi*time*f1*4),
                    #np.sin(2*np.pi*time*f1*6), np.cos(2*np.pi*time*f1*6),
                   ))).T
    Y2 = np.stack(((np.sin(2*np.pi*time*f2)  , np.cos(2*np.pi*time*f2),
                    np.sin(2*np.pi*time*f2*2), np.cos(2*np.pi*time*f2*2),
                    np.sin(2*np.pi*time*f2*4), np.cos(2*np.pi*time*f2*4),
                    #np.sin(2*np.pi*time*f2*6), np.cos(2*np.pi*time*f2*6),
                   ))).T
    Y3 = np.stack(((np.sin(2*np.pi*time*f3)  , np.cos(2*np.pi*time*f3),
                    np.sin(2*np.pi*time*f3*2), np.cos(2*np.pi*time*f3*2),
                    np.sin(2*np.pi*time*f3*4), np.cos(2*np.pi*time*f3*4),
                    #np.sin(2*np.pi*time*f3*6), np.cos(2*np.pi*time*f3*6),
                   ))).T

    # 
    r1 = CCA_corrcoeff(X, Y1)
    r2 = CCA_corrcoeff(X, Y2)
    r3 = CCA_corrcoeff(X, Y3)

    #
    L = X.shape[1]-1
    r1 = np.sort(r1)
    r2 = np.sort(r2)
    r3 = np.sort(r3)

    r = np.stack( np.mean((r1[L], r1[L-1])), np.mean((r2[L], r2[L-1])), np.mean((r3[L], r3[L-1])) )

    #print("r1: ", r1)
    #print("r2: ", r2)
    #print("r3: ", r3)
    
    return np.round(r, 7)

TypeError: 'numpy.float64' object is not iterable

In [13]:
r1

array([0.        , 0.        , 0.27588331])

In [17]:
r1[L-1]

0.0

In [18]:
np.mean((r3[L], r3[L-1]))

0.02218276357895923

In [19]:
np.mean((r2[L], r2[L-1]))

0.47518980418354934

In [22]:
r = np.stack(( np.mean((r1[L], r1[L-1])), np.mean((r2[L], r2[L-1])), np.mean((r3[L], r3[L-1])) ))